In [ ]:
import pandas as pd
import geopandas as gpd


In [ ]:
df_f = pd.read_csv('data/dane_szczepien.csv', sep=';',
                   encoding='utf-8')[['powiat_nazwa', 'wojewodztwo_nazwa']].groupby(['powiat_nazwa', 'wojewodztwo_nazwa']).sum().reset_index()


In [ ]:
# df = pd.read_csv('daneaktualne_szczepienia/20220123075316_rap_rcb_pow_szczepienia.csv',
#                  sep=';', encoding='windows-1250')
df_vacc = pd.read_csv('data/dane_szczepien.csv', sep=';',
                      encoding='utf-8').groupby('powiat_nazwa').sum().reset_index()
df_vacc = pd.merge(df_vacc, df_f, on='powiat_nazwa')
df_vacc_filtered = df_vacc[['powiat_nazwa', 'wojewodztwo_nazwa', 'powiat_teryt', 'liczba_ludnosci',
                            'w1_zaszczepieni_pacjenci', 'w3_zaszczepieni_pelna_dawka']]
df_distr = gpd.read_file("data/powiaty-max.geojson")

In [ ]:
df_distr['powiat_nazwa'] = df_distr['nazwa'].apply(lambda x: x.split(' ')[-1])
df_vacc_filtered['czesc_wyszczepienia'] = df_vacc_filtered['w3_zaszczepieni_pelna_dawka'] / df_vacc_filtered['liczba_ludnosci']


In [ ]:
print(df_distr.shape)
print(df_vacc_filtered.shape)


In [ ]:
full_df = pd.merge(df_distr, df_vacc_filtered, on='powiat_nazwa')


# Folium

In [ ]:
import folium 

In [ ]:
# geo_powiaty = "stance_detection_geotwitter/data/powiaty-max.geojson"
geo_powiaty = "data/powiaty-max.geojson"
with open(geo_powiaty, encoding="utf8") as file_powiaty:
    text_powiaty = file_powiaty.read()


In [ ]:
m = folium.Map(location=[52, 19.23], zoom_start=7, tiles='stamentoner')
folium.Choropleth(geo_data=text_powiaty,
    data=full_df,
    columns=['nazwa', 'czesc_wyszczepienia'],
    legend_name='Część wyszczepienia',
    fill_color='PuBu',
    fill_opacity=0.7,
    line_opacity=0.5,
    key_on='feature.properties.nazwa',
).add_to(m)

m


# Dane z AMC


In [ ]:
df_amc = pd.read_json(
    "data/location_data_all_v2.json")[['powiat_str', 'latitude', 'longitude', 'y3classes_sum', 'y3classes_str_general']]
df_amc['nazwa'] = df_amc['powiat_str']
df_amc.drop('powiat_str', axis=1)

df_amc = df_amc.groupby('nazwa').sum().reset_index()


In [ ]:
full_df_amc = pd.merge(df_amc, full_df, on='nazwa')

In [ ]:
full_df_amc = gpd.GeoDataFrame(full_df_amc)


# ML

In [ ]:
variable_names = [
    'y3classes_sum',
    'powiat_teryt',
    'liczba_ludnosci',
]


In [ ]:
from pysal.model import spreg


In [ ]:
# Fit OLS model
m1 = spreg.OLS(
    # Dependent variable
    full_df_amc[['czesc_wyszczepienia']].values,
    # Independent variables
    full_df_amc[variable_names].values,
    # Dependent variable name
    name_y='czesc_wyszczepienia',
    # Independent variable name
    name_x=variable_names
)


In [ ]:
print(m1.summary)


In [ ]:
from pysal.lib import weights
knn = weights.KNN.from_dataframe(full_df_amc, k=1)


In [ ]:
import seaborn

lag_residual = weights.spatial_lag.lag_spatial(knn, m1.u)
ax = seaborn.regplot(
    m1.u.flatten(),
    lag_residual.flatten(),
    line_kws=dict(color='orangered'),
    ci=None
)
ax.set_xlabel('Model Residuals - $u$')
ax.set_ylabel('Spatial Lag of Model Residuals - $W u$')


In [ ]:
from pysal.explore import esda
import contextily as cx
import matplotlib.pyplot as plt


# Re-weight W to 20 nearest neighbors
knn.reweight(k=20, inplace=True)
# Row standardise weights
knn.transform = 'R'
# Run LISA on residuals
outliers = esda.moran.Moran_Local(m1.u, knn, permutations=9999)
# Select only LISA cluster cores
error_clusters = (outliers.q % 2 == 1)
# Filter out non-significant clusters
error_clusters &= (outliers.p_sim <= .001)
# Add `error_clusters` and `local_I` columns
ax = full_df_amc.assign(
    error_clusters=error_clusters,
    local_I=outliers.Is
    # Retain error clusters only
).query(
    "error_clusters"
    # Sort by I value to largest plot on top
).sort_values(
    'local_I'
    # Plot I values
).plot(
    'local_I', cmap='bwr', marker='.', figsize=(16, 16)
)
# Add basemap
cx.add_basemap(ax, crs=full_df_amc.crs.to_string(), 
                source=cx.providers.Stamen.Watercolor, 
                zoom=8,
                interpolation='gaussian')
cx.add_basemap(ax, crs=full_df_amc.crs.to_string(), 
                source=cx.providers.Stamen.TonerLabels, zoom=8)

# # Remove axes
ax.set_axis_off()
plt.savefig('figure/localMoran.png', dpi=300)


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap
# import geopandas as gpd
# import pandas as pd
# from descartes import PolygonPatch
# from matplotlib.collections import PatchCollection

In [ ]:
# # fig = plt.figure(figsize=(8, 8))


# ax = full_df_amc.assign(
#     error_clusters=error_clusters,
#     local_I=outliers.Is
#     # Retain error clusters only
# ).query(
#     "error_clusters"
#     # Sort by I value to largest plot on top
# ).sort_values(
#     'local_I'
#     # Plot I values
# ).plot(
#     'local_I', cmap='bwr', marker='.', figsize=(16, 16))

# m = Basemap(resolution='i',
#             projection='aea',
#             ellps='WGS84', 
#             lat_0=52, lon_0=19.22,
#             width=0.75E6, height=0.7E6, ax = ax)
# # m.shadedrelief()


# m.drawcoastlines(linewidth=0.5)
# # m.drawcountries(linewidth=2, linestyle='solid', color='k' ) 
# # m.drawstates(linewidth=0.5, linestyle='solid', color='k')
# # water = 'lightskyblue'
# # earth = 'cornsilk'
# # m.drawcoastlines()
# # m.drawrivers(color=water, linewidth=1.5)
# # m.fillcontinents(
# #     color=earth,
# #     lake_color=water)
# # m.drawmapboundary(fill_color=water)
# # m.drawcountries()
# # m.drawmeridians(
# #     np.arange(-180, 180, 2), 
# #     labels=[False, False, False, True])
# # m.drawparallels(
# #     np.arange(0, 80), 
# #     labels=[True, True, False, False])


# plt.savefig('figure/mapa_general_regresion.png', dpi=400)

# Spatial ML

In [ ]:
import statsmodels.formula.api as sm
from shapely.geometry import Point


In [ ]:
# Czy jest w pobliżu najlepiej komentujących?
rk = weights.Rook.from_dataframe(full_df_amc)

full_df_amc['pos_neg_per_person'] = full_df_amc['y3classes_sum'] / full_df_amc['liczba_ludnosci']
full_df_amc['neighbouring_powiaty'] = [[full_df_amc.iloc[x]['powiat_nazwa'] for x in n_powiaty]
                                       for n_powiaty in list(rk.neighbors.values())]

most_pos = list(full_df_amc.sort_values('pos_neg_per_person', ascending=False).head(25)['powiat_nazwa'])
most_neg = list(full_df_amc.sort_values('pos_neg_per_person', ascending=True).head(25)['powiat_nazwa'])

def powiatowy(x):
    total = 0
    for y in x:
        if y in most_pos:
            total += 1
        elif y in most_neg:
            total -= 1
    return 1 if total > 0 else -1 if total < 0 else 0 

full_df_amc['neighborhood_pozytywny'] = full_df_amc['neighbouring_powiaty'].apply(powiatowy)


In [ ]:
full_df_amc.plot(column='neighborhood_pozytywny', categorical=False, legend=True, figsize=(12,12))


In [ ]:
# variable_names.append('neighborhood_pozytywny')
variable_names.append('pos_neg_per_person')

### Po dodaniu województw jako zależności geograficznych.

In [ ]:
# PySAL spatial fixed effect implementation
m4 = spreg.OLS_Regimes(
    # Dependent variable
    full_df_amc[['czesc_wyszczepienia']].values,
    # Independent variables
    full_df_amc[variable_names].values,
    # Variable specifying neighborhood membership
    full_df_amc['wojewodztwo_nazwa'].tolist(),
    # Allow the constant term to vary by group/regime
    constant_regi='many',
    # Variables to be allowed to vary (True) or kept
    # constant (False). Here we set all to False
    cols2regi=[False]*len(variable_names),
    # Allow separate sigma coefficients to be estimated
    # by regime (False so a single sigma)
    regime_err_sep=False,
    # Dependent variable name
    name_y='czesc_wyszczepienia',
    # Independent variables names
    name_x=variable_names
)


In [ ]:
print(m4.summary)


In [ ]:
knn = weights.KNN.from_dataframe(full_df_amc, k=1)
lag_residual = weights.spatial_lag.lag_spatial(knn, m4.u)
ax = seaborn.regplot(
    m4.u.flatten(),
    lag_residual.flatten(),
    line_kws=dict(color='orangered'),
    ci=None
)
ax.set_xlabel('Model Residuals - $u$')
ax.set_ylabel('Spatial Lag of Model Residuals - $W u$')


### Po sprawdzeniu "najbardziej pozytywnych i negatywnych" sąsiadów jako zależności geo

In [ ]:
# PySAL spatial regimes implementation
m5 = spreg.OLS_Regimes(
    # Dependent variable
    full_df_amc[['czesc_wyszczepienia']].values,
    # Independent variables
    full_df_amc[variable_names].values,
    # Variable specifying neighborhood membership
    full_df_amc['neighborhood_pozytywny'].tolist(),
    # Allow the constant term to vary by group/regime
    constant_regi='many',
    # Allow separate sigma coefficients to be estimated
    # by regime (False so a single sigma)
    regime_err_sep=False,
    # Dependent variable name
    name_y='czesc_wyszczepienia',
    # Independent variables names
    name_x=variable_names
)


In [ ]:
print(m5.summary)


### The SLX Model - dodanie spacial lag'a

In [ ]:
wx = full_df_amc[variable_names].apply(
    lambda y: weights.spatial_lag.lag_spatial(knn, y)
    # Rename the spatial lag, adding w_ to the original name
).rename(columns=lambda c: 'w_'+c
    # Remove the lag of the binary variable for apartments
         )

slx_exog = full_df_amc[variable_names].join(wx)
# Fit linear model with `spreg`
m6 = spreg.OLS(
    # Dependent variable
    full_df_amc[['czesc_wyszczepienia']].values,
    # Independent variables
    slx_exog.values,
    # Dependent variable name
    name_y='czesc_wyszczepienia',
    # Independent variables names
    name_x=slx_exog.columns.tolist()
)


In [ ]:
print(m6.summary)


# All together!

In [ ]:
full_df_amc = pd.merge(df_amc, full_df, on='nazwa')
full_df_amc = gpd.GeoDataFrame(full_df_amc)

variable_names = [
    'y3classes_sum',
    'powiat_teryt',
    'liczba_ludnosci',
]

rk = weights.Rook.from_dataframe(full_df_amc)

full_df_amc['pos_neg_per_person'] = full_df_amc['y3classes_sum'] / full_df_amc['liczba_ludnosci']
full_df_amc['neighbouring_powiaty'] = [[full_df_amc.iloc[x]['powiat_nazwa'] for x in n_powiaty]
                                       for n_powiaty in list(rk.neighbors.values())]

most_pos = list(full_df_amc.sort_values('pos_neg_per_person', ascending=False).head(25)['powiat_nazwa'])
most_neg = list(full_df_amc.sort_values('pos_neg_per_person', ascending=True).head(25)['powiat_nazwa'])


def powiatowy(x):
    total = 0
    for y in x:
        if y in most_pos:
            total += 1
        elif y in most_neg:
            total -= 1
    return 1 if total > 0 else -1 if total < 0 else 0


full_df_amc['neighborhood_pozytywny'] = full_df_amc['neighbouring_powiaty'].apply(powiatowy)

variable_names.append('neighborhood_pozytywny') # Adding this info as a var
variable_names.append('pos_neg_per_person')

knn = weights.KNN.from_dataframe(full_df_amc, k=20)
wx = full_df_amc[variable_names].apply(
    lambda y: weights.spatial_lag.lag_spatial(knn, y)
    # Rename the spatial lag, adding w_ to the original name
).rename(columns=lambda c: 'w_'+c
         # Remove the lag of the binary variable for apartments
         )

slx_exog = full_df_amc[variable_names].join(wx)


In [ ]:
slx_exog.iloc[0]


In [ ]:
m7 = spreg.OLS_Regimes(
    # Dependent variable
    full_df_amc[['czesc_wyszczepienia']].values,
    # Independent variables
    slx_exog.values,
    # Variable specifying neighborhood membership
    full_df_amc['wojewodztwo_nazwa'].tolist(),
    # Allow the constant term to vary by group/regime
    constant_regi='many',
    # Variables to be allowed to vary (True) or kept
    # constant (False). Here we set all to False
    cols2regi=[False]*len(variable_names)*2,
    # Allow separate sigma coefficients to be estimated
    # by regime (False so a single sigma)
    regime_err_sep=False,
    # Dependent variable name
    name_y='czesc_wyszczepienia',
    # Independent variables names
    name_x=slx_exog.columns.tolist()
)


In [ ]:
print(m7.summary)


# TEST AREA